In [0]:
!git clone https://github.com/dipsankarb/graphs.git

Cloning into 'graphs'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 0), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark


--2019-11-14 06:29:05--  http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
Resolving apachemirror.wuchna.com (apachemirror.wuchna.com)... 104.26.2.179, 104.26.3.179, 2606:4700:20::681a:2b3, ...
Connecting to apachemirror.wuchna.com (apachemirror.wuchna.com)|104.26.2.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230091034 (219M) [application/x-gzip]
Saving to: ‘spark-2.4.4-bin-hadoop2.7.tgz’

spark-2.4.4-bin-had 100%[===================>] 219.43M  7.91MB/s    in 29s     

2019-11-14 06:29:34 (7.67 MB/s) - ‘spark-2.4.4-bin-hadoop2.7.tgz’ saved [230091034/230091034]



In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
import re

def computeContribs(urls, rank):
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls):
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [0]:
!rm -r /content/output_page_rank_small

In [0]:
lines = spark.read.text('/content/graphs/small.txt').rdd.map(lambda r: r[0])
links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()


ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

N = ranks.count()
iterations = 40

for iteration in range(iterations):
    contribs = links.join(ranks).flatMap(lambda urls_rank: computeContribs(urls_rank[1][0], urls_rank[1][1]))
    
    ranks = contribs.reduceByKey(lambda r1,r2 : r1 + r2).mapValues(lambda rank: rank * 0.80 + 0.20*(1/(N)))


rank_list = {}
for (link, rank) in ranks.collect():
    rank_list[link] = rank
    print("%s has rank: %s." % (link, rank))

spark.stop()

48 has rank: 0.025818524132253773.
87 has rank: 0.013665084070648646.
72 has rank: 0.00440210804971127.
9 has rank: 0.013049811484297276.
41 has rank: 0.008139889487303407.
1 has rank: 0.030471828510931123.
75 has rank: 0.006714930274172718.
95 has rank: 0.007706722013793672.
76 has rank: 0.004905361640649055.
58 has rank: 0.005188754288486331.
88 has rank: 0.0046405896088193.
96 has rank: 0.018889370810830648.
25 has rank: 0.007471022397910645.
84 has rank: 0.0040628361847872795.
34 has rank: 0.006446306841456598.
70 has rank: 0.010623550555850134.
31 has rank: 0.012489912832965118.
26 has rank: 0.014542755631943509.
78 has rank: 0.015024727630785389.
79 has rank: 0.01991980448975001.
97 has rank: 0.0063588445866225125.
3 has rank: 0.007874568376559015.
83 has rank: 0.013447090705256048.
99 has rank: 0.009116845770037445.
35 has rank: 0.013171253462686099.
68 has rank: 0.005091070030892512.
100 has rank: 0.013122901884215584.
22 has rank: 0.015497518134823772.
23 has rank: 0.004159062

In [0]:
ranks = dict(sorted(rank_list.items(), key=lambda kv: kv[1]))

x = 0
print('Bottom 5 : \nNode\t Rank')
for k in ranks:
    x += 1
    print(k, ranks[k])
    if(x == 5):
        break

Bottom 5 : 
Node	 Rank
558 0.00034989710662642324
93 0.0003760331380601572
62 0.00037620234553268977
424 0.00038010454268700593
408 0.000417344143686734


In [0]:
ranks = dict(sorted(rank_list.items(), key=lambda kv: kv[1],reverse = True))

x = 0
print('Top 5 : \nNode\t Rank')
for k in ranks:
    x += 1
    print(k, ranks[k])
    if(x == 5):
        break

Top 5 : 
Node	 Rank
263 0.002325110013647295
537 0.002251117718147551
965 0.0022161262486851932
243 0.002131150723863407
285 0.002097595272978842


In [0]:
lines = spark.read.text('/content/graphs/whole.txt').rdd.map(lambda r: r[0])
links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()

ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

N = ranks.count()
iterations = 40

for iteration in range(iterations):
    contribs = links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
    ranks = contribs.reduceByKey(lambda r1,r2 : r1 + r2).mapValues(lambda rank: rank * 0.80 + 0.20*(1/(N)))

rank_list = {}
for (link, rank) in ranks.collect():
    rank_list[link] = rank
    # print("%s has rank: %s." % (link, rank))

# ranks.collect()
# ranks.saveAsTextFile('output_page_rank_large')
spark.stop()